# Лабораторная работа №7 (Проведение исследований моделями семантической сегментации)

### Выбор набора данных и обоснование  
Набор данных: [Leaf Disease Segmentation Dataset](https://www.kaggle.com/datasets/fakhrealam9537/leaf-disease-segmentation-dataset) на Kaggle.  

Обоснование выбора:  
Этот датасет содержит изображения листьев растений с размеченными областями поражений, вызванных болезнями. Он подходит для задачи семантической сегментации, так как позволяет модели локализовать и классифицировать точные области повреждений на листьях. Автоматизация диагностики заболеваний растений в сельском хозяйстве.

### Метрики качества и их обоснование  
Для оценки моделей семантической сегментации в этой задаче предлагаются следующие метрики:  

1. IoU (Intersection over Union)  
   - Метрика напрямую оценивает точность сегментации, измеряя степень перекрытия между предсказанными и реальными областями поражений. Она устойчива к несбалансированным классам (например, когда здоровая ткань занимает большую площадь, чем пораженная).  

2. Dice Coefficient  
   - Хорошо подходит для задач с небольшими объектами (например, маленькими пятнами на листьях) и чувствителен к точности воспроизведения формы поражения.  

3. Accuracy  
   - Простая метрика для оценки общей точности, но она может быть обманчивой при дисбалансе (например, если здоровая ткань составляет 90% изображения).  

In [1]:
!pip install segmentation_models_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
import kagglehub
import torch
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.utils.metrics import IoU, Accuracy, Fscore
from segmentation_models_pytorch.utils.losses import DiceLoss
from segmentation_models_pytorch.losses import JaccardLoss
from torch.optim import Adam
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.nn import BCEWithLogitsLoss
import torch.nn as nn

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [4]:
dataset_path = kagglehub.dataset_download("fakhrealam9537/leaf-disease-segmentation-dataset")

In [5]:
# Пути к данным (изображения и соответствующие маски)
images_dir = "/kaggle/input/leaf-disease-segmentation-dataset/aug_data/aug_data/images"
masks_dir = "/kaggle/input/leaf-disease-segmentation-dataset/aug_data/aug_data/masks"

# Гиперпараметры модели
BATCH_SIZE = 8
EPOCHS = 5
LR = 1e-4

# Аугментации для обучающей выборки
train_transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),  # Добавлено для борьбы с дисбалансом
    A.ToFloat(max_value=255.0),
    ToTensorV2()
])

# Аугментации для валидационной выборки
val_transform = A.Compose([
    A.Resize(256, 256),
    A.ToFloat(max_value=255.0),
    ToTensorV2()
])

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
<ipython-input-5-e7af24bce481>:16: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.3),  # Добавлено для борьбы с дисбалансом


In [6]:
class LeafDiseaseDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.image_names = os.listdir(images_dir)

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        image_path = os.path.join(self.images_dir, image_name)
        mask_name = os.path.splitext(image_name)[0] + ".png"
        mask_path = os.path.join(self.masks_dir, mask_name)

        # Проверка существования
        if not os.path.exists(mask_path):
            raise FileNotFoundError(f"Mask not found: {mask_path}")

        # Загрузка изображения и маски
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, 0)  # ЧБ маска

        # Проверка успешной загрузки
        if image is None:
            raise ValueError(f"Failed to load image: {image_path}")
        if mask is None:
            raise ValueError(f"Failed to load mask: {mask_path}")

        # Применение трансформаций
        if self.transform:
            transformed = self.transform(image=image, mask=mask)
            image = transformed["image"]  # Уже float32
            mask = transformed["mask"].float()  # Уже float32

        # Нормализация маски в [0,1]
        mask = mask / 255.0
        mask = mask.unsqueeze(0)

        return image, mask

# Загрузка данных
train_dataset = LeafDiseaseDataset(images_dir, masks_dir, transform=train_transform)
val_dataset = LeafDiseaseDataset(images_dir, masks_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [7]:
# Создание модели U-Net с ResNet50 в качестве энкодера
model_unet = smp.Unet(
    encoder_name="resnet50",
    encoder_weights="imagenet",
    classes=1,
    activation=None,
    in_channels=3
).to(DEVICE)

# Создание модели DeepLabV3+ с MIT-B2 как энкодером
model_deeplab = smp.DeepLabV3Plus(
    encoder_name="mit_b2",
    encoder_weights="imagenet",
    classes=1,
    activation=None,
    in_channels=3
).to(DEVICE)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/98.9M [00:00<?, ?B/s]

In [8]:
pos_weight = torch.tensor([10.0]).to(DEVICE)  # Подбирается в зависимости от дисбаланса
loss_fn = {
    "bce": torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight),
    "dice": smp.losses.DiceLoss(mode="binary")
}

optimizer_unet = Adam(model_unet.parameters(), lr=LR)
optimizer_deeplab = Adam(model_deeplab.parameters(), lr=LR)

# Метрики
metrics = {
    "iou": smp.utils.metrics.IoU(threshold=0.5),
    "accuracy": smp.utils.metrics.Accuracy(threshold=0.5),
    "dice": smp.utils.metrics.Fscore(beta=1.0, threshold=0.5)
}

# Обучение
def train_model(model, optimizer, train_loader, val_loader, epochs=10, model_name="unet"):
    best_iou = 0
    history = {"train_loss": [], "val_loss": [], "val_iou": [], "val_accuracy": [], "val_dice": []}

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(images)
            output_sigmoid = torch.sigmoid(outputs)
            loss = loss_fn["bce"](outputs, masks) + loss_fn["dice"](output_sigmoid, masks)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Валидация
        model.eval()
        val_loss = 0
        iou, accuracy, dice = 0, 0, 0
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(DEVICE)
                masks = masks.to(DEVICE)
                outputs = model(images)
                output_sigmoid = torch.sigmoid(outputs)
                loss = loss_fn["bce"](outputs, masks) + loss_fn["dice"](output_sigmoid, masks)
                val_loss += loss.item()

                # Метрики
                iou += metrics["iou"](output_sigmoid, masks).item()
                accuracy += metrics["accuracy"](output_sigmoid, masks).item()
                dice += metrics["dice"](output_sigmoid, masks).item()

        # Логгирование
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        avg_iou = iou / len(val_loader)
        avg_accuracy = accuracy / len(val_loader)
        avg_dice = dice / len(val_loader)

        history["train_loss"].append(avg_train_loss)
        history["val_loss"].append(avg_val_loss)
        history["val_iou"].append(avg_iou)
        history["val_accuracy"].append(avg_accuracy)
        history["val_dice"].append(avg_dice)

        print(f"\nEpoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        print(f"IoU: {avg_iou:.4f} | Accuracy: {avg_accuracy:.4f} | Dice: {avg_dice:.4f}")

        # Сохранение лучшей модели
        if avg_iou > best_iou:
            best_iou = avg_iou
            torch.save(model.state_dict(), f"{model_name}_best.pth")
    return history

In [ ]:
# Обучение Unet
print("Training Unet")
history_unet = train_model(model_unet, optimizer_unet, train_loader, val_loader, EPOCHS, model_name="unet")

Training Unet...


Epoch 1/5: 100%|██████████| 368/368 [01:39<00:00,  3.69it/s]



Epoch 1 | Train Loss: 1.3700 | Val Loss: 1.2945
IoU: 0.1137 | Accuracy: 0.8125 | Dice: 0.2038


Epoch 2/5: 100%|██████████| 368/368 [01:37<00:00,  3.79it/s]



Epoch 2 | Train Loss: 1.2798 | Val Loss: 1.2505
IoU: 0.1249 | Accuracy: 0.8199 | Dice: 0.2220


...


Epoch 3 | Train Loss: 1.2603 | Val Loss: 1.2307
IoU: 0.1294 | Accuracy: 0.8249 | Dice: 0.2291


Epoch 4/5: 100%|██████████| 368/368 [01:36<00:00,  3.80it/s]



Epoch 4 | Train Loss: 1.2485 | Val Loss: 1.2243
IoU: 0.1297 | Accuracy: 0.8236 | Dice: 0.2295


Epoch 5/5: 100%|██████████| 368/368 [01:37<00:00,  3.77it/s]



Epoch 5 | Train Loss: 1.2355 | Val Loss: 1.2191
IoU: 0.1306 | Accuracy: 0.8242 | Dice: 0.2310


In [ ]:
# Обучение DeepLabV3+
print("Training DeepLabV3+")
history_deeplab = train_model(model_deeplab, optimizer_deeplab, train_loader, val_loader, EPOCHS, model_name="deeplab")

Training DeepLabV3+...


Epoch 1/5: 100%|██████████| 368/368 [01:57<00:00,  3.13it/s]



Epoch 1 | Train Loss: 1.3806 | Val Loss: 1.2587
IoU: 0.1276 | Accuracy: 0.8280 | Dice: 0.2263


Epoch 2/5: 100%|██████████| 368/368 [01:56<00:00,  3.15it/s]



Epoch 2 | Train Loss: 1.2577 | Val Loss: 1.2343
IoU: 0.1319 | Accuracy: 0.8298 | Dice: 0.2331


...


Epoch 3 | Train Loss: 1.2369 | Val Loss: 1.2224
IoU: 0.1295 | Accuracy: 0.8225 | Dice: 0.2293


Epoch 4/5: 100%|██████████| 368/368 [01:56<00:00,  3.15it/s]



Epoch 4 | Train Loss: 1.2340 | Val Loss: 1.2149
IoU: 0.1322 | Accuracy: 0.8258 | Dice: 0.2335


Epoch 5/5: 100%|██████████| 368/368 [01:56<00:00,  3.16it/s]



Epoch 5 | Train Loss: 1.2228 | Val Loss: 1.2230
IoU: 0.1263 | Accuracy: 0.8175 | Dice: 0.2242


###	Улучшение бейзлайна
Аугментации данных улучшат обобщающую способность модели
Добавление аугментаций, таких как RandomBrightnessContrast, ShiftScaleRotate, и CoarseDropout, поможет модели лучше адаптироваться к различным условиям освещения и повреждениям, увеличивая разнообразие данных.

In [9]:
train_transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=30, p=0.7),
    A.CoarseDropout(max_holes=12, max_height=64, max_width=64, p=0.5),
    A.ToFloat(max_value=255.0),
    ToTensorV2()
])

train_dataset = LeafDiseaseDataset(images_dir, masks_dir, transform=train_transform)
val_dataset = LeafDiseaseDataset(images_dir, masks_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

<ipython-input-9-e03c5c638eff>:7: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=12, max_height=64, max_width=64, p=0.5),


In [ ]:
# Обучение U-Net с аугментациями
print("Training Unet with augmentations")
history_unet_aug = train_model(model_unet, optimizer_unet, train_loader, val_loader, EPOCHS, model_name="unet_aug")

Training Unet with augmentations...


Epoch 1/5: 100%|██████████| 368/368 [01:39<00:00,  3.71it/s]



Epoch 1 | Train Loss: 1.2432 | Val Loss: 1.2288
IoU: 0.1320 | Accuracy: 0.8308 | Dice: 0.2331


Epoch 2/5: 100%|██████████| 368/368 [01:38<00:00,  3.73it/s]



Epoch 2 | Train Loss: 1.2410 | Val Loss: 1.2223
IoU: 0.1295 | Accuracy: 0.8223 | Dice: 0.2293


...


Epoch 3 | Train Loss: 1.2310 | Val Loss: 1.2251
IoU: 0.1273 | Accuracy: 0.8192 | Dice: 0.2256


Epoch 4/5: 100%|██████████| 368/368 [01:40<00:00,  3.68it/s]



Epoch 4 | Train Loss: 1.2287 | Val Loss: 1.2223
IoU: 0.1317 | Accuracy: 0.8287 | Dice: 0.2327


Epoch 5/5: 100%|██████████| 368/368 [01:38<00:00,  3.75it/s]



Epoch 5 | Train Loss: 1.2209 | Val Loss: 1.2136
IoU: 0.1304 | Accuracy: 0.8231 | Dice: 0.2306


In [ ]:
# Обучение DeepLabV3+ с аугментациями
print("Training DeepLabV3+")
history_deeplab = train_model(model_deeplab, optimizer_deeplab, train_loader, val_loader, EPOCHS, model_name="deeplab")

Training DeepLabV3+...


Epoch 1/5: 100%|██████████| 368/368 [01:58<00:00,  3.12it/s]



Epoch 1 | Train Loss: 1.2126 | Val Loss: 1.2076
IoU: 0.1351 | Accuracy: 0.8294 | Dice: 0.2380


Epoch 2/5: 100%|██████████| 368/368 [01:57<00:00,  3.14it/s]



Epoch 2 | Train Loss: 1.2062 | Val Loss: 1.2041
IoU: 0.1345 | Accuracy: 0.8276 | Dice: 0.2370


...


Epoch 3 | Train Loss: 1.2041 | Val Loss: 1.2030
IoU: 0.1364 | Accuracy: 0.8307 | Dice: 0.2400


Epoch 4/5: 100%|██████████| 368/368 [02:02<00:00,  3.00it/s]



Epoch 4 | Train Loss: 1.2157 | Val Loss: 1.2081
IoU: 0.1332 | Accuracy: 0.8265 | Dice: 0.2350


Epoch 5/5: 100%|██████████| 368/368 [01:57<00:00,  3.12it/s]



Epoch 5 | Train Loss: 1.2047 | Val Loss: 1.2005
IoU: 0.1360 | Accuracy: 0.8290 | Dice: 0.2394


### Своя имплементация алгоритма

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class UNetWithResNet50(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, pretrained=True):
        super(UNetWithResNet50, self).__init__()

        # Загружаем предобученный ResNet50 и берем только encoder часть
        resnet = models.resnet50(pretrained=pretrained)

        # Сохраняем начальные слои ResNet
        self.encoder = nn.ModuleList([
            nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool),
            resnet.layer1,
            resnet.layer2,
            resnet.layer3,
            resnet.layer4,
        ])

        # Декодер
        self.up4 = self._up_block(2048, 1024)
        self.up3 = self._up_block(1024, 512)
        self.up2 = self._up_block(512, 256)
        self.up1 = self._up_block(256, 64)

        self.final_conv = nn.Sequential(
            nn.Conv2d(64, out_channels, kernel_size=1),
        )

    def _up_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        skips = []
        for enc in self.encoder:
            x = enc(x)
            skips.append(x)

        d4 = self.up4(skips[4])
        d4 = d4 + skips[3]

        d3 = self.up3(d4)
        d3 = d3 + skips[2]

        d2 = self.up2(d3)
        d2 = d2 + skips[1]

        d1 = self.up1(d2)
        d1 = d1 + F.interpolate(skips[0], size=d1.shape[2:], mode='bilinear', align_corners=False)

        out = self.final_conv(d1)

        if out.shape[-2:] != (256, 256):
            out = F.interpolate(out, size=(256, 256), mode='bilinear', align_corners=False)

        return out

In [11]:
model = UNetWithResNet50(in_channels=3, out_channels=1).to(DEVICE)
# Оптимизатор Adam с заданной скоростью обучения
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

print("Training U-Net with ResNet50 encoder")
history = train_model(model, optimizer, train_loader, val_loader, EPOCHS)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 63.6MB/s]


Training U-Net with ResNet50 encoder...


Epoch 1/5: 100%|██████████| 368/368 [02:23<00:00,  2.57it/s]



Epoch 1 | Train Loss: 1.3561 | Val Loss: 1.2882
IoU: 0.1146 | Accuracy: 0.8184 | Dice: 0.2054


Epoch 2/5: 100%|██████████| 368/368 [01:48<00:00,  3.39it/s]


...


Epoch 3 | Train Loss: 1.2496 | Val Loss: 1.2328
IoU: 0.1281 | Accuracy: 0.8271 | Dice: 0.2270


Epoch 4/5: 100%|██████████| 368/368 [01:53<00:00,  3.24it/s]



Epoch 4 | Train Loss: 1.2393 | Val Loss: 1.2291
IoU: 0.1292 | Accuracy: 0.8276 | Dice: 0.2287


Epoch 5/5: 100%|██████████| 368/368 [01:47<00:00,  3.41it/s]



Epoch 5 | Train Loss: 1.2373 | Val Loss: 1.2256
IoU: 0.1306 | Accuracy: 0.8284 | Dice: 0.2310


### Улучшенный бейзлайн

In [12]:
train_transform = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=30, p=0.7),
    A.CoarseDropout(max_holes=12, max_height=64, max_width=64, p=0.5),
    A.ToFloat(max_value=255.0),
    ToTensorV2()
])

train_dataset = LeafDiseaseDataset(images_dir, masks_dir, transform=train_transform)
val_dataset = LeafDiseaseDataset(images_dir, masks_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

<ipython-input-12-e03c5c638eff>:7: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=12, max_height=64, max_width=64, p=0.5),


In [13]:
model = UNetWithResNet50(in_channels=3, out_channels=1).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

print("Training custom U-Net with ResNet50 encoder")
history = train_model(model, optimizer, train_loader, val_loader, EPOCHS)

Training U-Net with ResNet50 encoder...


Epoch 1/5: 100%|██████████| 368/368 [01:47<00:00,  3.42it/s]



Epoch 1 | Train Loss: 1.3543 | Val Loss: 1.2867
IoU: 0.1152 | Accuracy: 0.8246 | Dice: 0.2063


Epoch 2/5: 100%|██████████| 368/368 [01:49<00:00,  3.36it/s]



Epoch 2 | Train Loss: 1.2782 | Val Loss: 1.2499
IoU: 0.1232 | Accuracy: 0.8241 | Dice: 0.2193


...


Epoch 3 | Train Loss: 1.2516 | Val Loss: 1.2420
IoU: 0.1241 | Accuracy: 0.8195 | Dice: 0.2207


Epoch 4/5: 100%|██████████| 368/368 [01:47<00:00,  3.42it/s]



Epoch 4 | Train Loss: 1.2418 | Val Loss: 1.2371
IoU: 0.1219 | Accuracy: 0.8114 | Dice: 0.2171


Epoch 5/5: 100%|██████████| 368/368 [01:47<00:00,  3.43it/s]



Epoch 5 | Train Loss: 1.2359 | Val Loss: 1.2267
IoU: 0.1307 | Accuracy: 0.8285 | Dice: 0.2311


В улучшенном бейзлайне метрики улучшились

| Модель | Train Loss | Val Loss | IoU | Accuracy | Dice |
|--------|------------|----------|-------|-------------|---------|
| U-Net | 1.2355 | 1.2191 | 0.1306 | 0.8242 | 0.2310 |
| DeepLabV3+ | 1.2228 | 1.2230 | 0.1322 | 0.8298 | 0.2335 |
| U-Net + Augmentations | 1.2209 | 1.2136 | 0.1317 | 0.8308 | 0.2327 |
| DeepLabV3+ + Augmentations | 1.2047 | 1.2005 | 0.1364 | 0.8307 | 0.2400 |
| Custom U-Net (ResNet50) | 1.2373 | 1.2256 | 0.1306 | 0.8284 | 0.2310 |
| Custom U-Net + Augmentations | 1.2359 | 1.2267 | 0.1307 | 0.8285 | 0.2311 |